# Feature Engineering

---

1. Import packages
2. Load data
3. Feature engineering

---

## 1. Import packages

In [19]:
import pandas as pd
import numpy as np

---
## 2. Load data

In [3]:
df = pd.read_csv('./clean_data_after_eda.csv')
df["date_activ"] = pd.to_datetime(df["date_activ"], format='%Y-%m-%d')
df["date_end"] = pd.to_datetime(df["date_end"], format='%Y-%m-%d')
df["date_modif_prod"] = pd.to_datetime(df["date_modif_prod"], format='%Y-%m-%d')
df["date_renewal"] = pd.to_datetime(df["date_renewal"], format='%Y-%m-%d')

In [4]:
df.head(6)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.000131,4.100838e-05,9.084737e-04,2.086294,99.530517,44.235794,2.086425,9.953056e+01,4.423670e+01,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000003,1.217891e-03,0.000000e+00,0.009482,0.000000,0.000000,0.009485,1.217891e-03,0.000000e+00,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000004,9.450150e-08,0.000000e+00,0.000000,0.000000,0.000000,0.000004,9.450150e-08,0.000000e+00,0
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,...,0.000003,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000003,0.000000e+00,0.000000e+00,0
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,...,0.000011,2.896760e-06,4.860000e-10,0.000000,0.000000,0.000000,0.000011,2.896760e-06,4.860000e-10,0
5,1aa498825382410b098937d65c4ec26d,usilxuppasemubllopkaafesmlibmsdf,8302,0,1998,2011-12-09,2016-12-09,2015-11-01,2015-12-10,796.94,...,0.000010,4.240524e-06,0.000000e+00,0.000000,0.000000,0.000000,0.000010,4.240524e-06,0.000000e+00,1


---

## 3. Feature engineering

### Difference between off-peak prices in December and preceding January

Below is the code created by your colleague to calculate the feature described above. Use this code to re-create this feature and then think about ways to build on this feature to create features with a higher predictive power.

In [5]:
price_df = pd.read_csv('price_data.csv')
price_df["price_date"] = pd.to_datetime(price_df["price_date"], format='%Y-%m-%d')
price_df.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


In [6]:
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()

# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'}), jan_prices.drop(columns='price_date'), on='id')
diff['offpeak_diff_dec_january_energy'] = diff['dec_1'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_january_power'] = diff['dec_2'] - diff['price_off_peak_fix']
diff = diff[['id', 'offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power']]
diff.head()

,id,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916
4,00114d74e963e47177db89bc70108537,-0.003994,-0.000001


Now it is time to get creative and to conduct some of your own feature engineering! Have fun with it, explore different ideas and try to create as many as you can!

In [7]:
# 1. Group by 'id' and find the max and min of the monthly prices
max_min_prices = monthly_price_by_id.groupby('id').agg({
    'price_off_peak_var': ['max', 'min'],
    'price_off_peak_fix': ['max', 'min']
}).reset_index()

# 2. Flatten the multi-index columns
max_min_prices.columns = ['id', 'var_max', 'var_min', 'fix_max', 'fix_min']

# 3. Calculate the maximum spread (volatility)
max_min_prices['max_change_energy'] = max_min_prices['var_max'] - max_min_prices['var_min']
max_min_prices['max_change_power'] = max_min_prices['fix_max'] - max_min_prices['fix_min']

# 4. Keep only the necessary columns
max_diff_features = max_min_prices[['id', 'max_change_energy', 'max_change_power']]
max_diff_features.head()

,id,max_change_energy,max_change_power
0,0002203ffbb812588b632b9e628cc38d,0.008161,0.162916
1,0004351ebdd665e6ee664792efc4fd13,0.004462,0.177779
2,0010bcc39e42b3c2131ed2ce55246e3c,0.054905,1.500000
3,0010ee3855fdea87602a5b7aba8e42de,0.010018,0.162916
4,00114d74e963e47177db89bc70108537,0.004462,0.000001


In [8]:
# Max, Min, and Mean for each ID
stats = monthly_price_by_id.groupby('id').agg({
    'price_off_peak_var': ['max', 'min', 'mean'],
    'price_off_peak_fix': ['max', 'min', 'mean']
}).reset_index()

# Flatten Columns
stats.columns = ['id', 'v_max', 'v_min', 'v_mean', 'f_max', 'f_min', 'f_mean']

# Normalized Range (Relative Volatility)
# (Max - Min) / Mean
stats['norm_max_change_energy'] = (stats['v_max'] - stats['v_min']) / stats['v_mean']
stats['norm_max_change_power'] = (stats['f_max'] - stats['f_min']) / stats['f_mean']

# Handle potential division by zero (if mean is 0)
stats[['norm_max_change_energy', 'norm_max_change_power']] = stats[['norm_max_change_energy', 'norm_max_change_power']].fillna(0)

# Clean up the final dataframe
normalized_diff = stats[['id', 'norm_max_change_energy', 'norm_max_change_power']]
normalized_diff.head()

,id,norm_max_change_energy,norm_max_change_power
0,0002203ffbb812588b632b9e628cc38d,0.065635,4.002670e-03
1,0004351ebdd665e6ee664792efc4fd13,0.030473,4.005340e-03
2,0010bcc39e42b3c2131ed2ce55246e3c,0.302409,3.309818e-02
3,0010ee3855fdea87602a5b7aba8e42de,0.084357,4.008017e-03
4,00114d74e963e47177db89bc70108537,0.030164,2.710827e-08


In [13]:
# 1. Get the January (starting) prices for each ID
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
jan_prices = jan_prices.rename(columns={
    'price_off_peak_var': 'jan_var',
    'price_off_peak_fix': 'jan_fix'
})[['id', 'jan_var', 'jan_fix']]

# 2. Get the Maximum price reached at any point in the year
max_prices = monthly_price_by_id.groupby('id').agg({
    'price_off_peak_var': 'max',
    'price_off_peak_fix': 'max'
}).reset_index()
max_prices.columns = ['id', 'max_var', 'max_fix']

# 3. Merge and calculate the "Unfavorable Shock"
unfavorable_diff = pd.merge(max_prices, jan_prices, on='id')

# Normalized: (Max reached - January Price) / January Price
# This gives the maximum percentage increase a customer experienced.
unfavorable_diff['max_unfavorable_shk_var'] = (unfavorable_diff['max_var'] - unfavorable_diff['jan_var']) / unfavorable_diff['jan_var']
unfavorable_diff['max_unfavorable_shk_fix'] = (unfavorable_diff['max_fix'] - unfavorable_diff['jan_fix']) / unfavorable_diff['jan_fix']

# 4. Clean up
# If price only went down, the value will be negative. 
unfavorable_diff['max_unfavorable_shk_var'] = unfavorable_diff['max_unfavorable_shk_var'].clip(lower=0).fillna(0)
unfavorable_diff['max_unfavorable_shk_fix'] = unfavorable_diff['max_unfavorable_shk_fix'].clip(lower=0).fillna(0)

unfavorable_features = unfavorable_diff[['id', 'max_unfavorable_shk_var', 'max_unfavorable_shk_fix']]
unfavorable_features.head()

,id,max_unfavorable_shk_var,max_unfavorable_shk_fix
0,0002203ffbb812588b632b9e628cc38d,0.015615,0.004016
1,0004351ebdd665e6ee664792efc4fd13,0.002418,0.004016
2,0010bcc39e42b3c2131ed2ce55246e3c,0.364002,0.033750
3,0010ee3855fdea87602a5b7aba8e42de,0.000000,0.004016
4,00114d74e963e47177db89bc70108537,0.003132,0.000000


In [20]:
# 1. Create a deep copy to avoid SettingWithCopy warnings
client_df = df.copy()

# 2. Consumption Shock (Last Month vs. Monthly Average)
# We divide by 12 to get the monthly average from the 12m total
client_df['cons_monthly_avg'] = client_df['cons_12m'] / 12
client_df['consumption_shock_12m'] = client_df['cons_last_month'] / client_df['cons_monthly_avg']

# 3. Forecast Gap (Expectation vs. Reality)
# Measures how much the actual consumption deviated from the forecast
# A large positive gap means they are using much less than expected.
client_df['forecast_gap_energy'] = (client_df['forecast_cons_12m'] - client_df['cons_12m']) / client_df['forecast_cons_12m']

# 4. Handle Infinity/NaNs (caused by zero-consumption customers)
# We fill with 0 to represent "No Change/No Gap" for those edge cases
cols_to_fix = ['consumption_shock_12m', 'forecast_gap_energy']
client_df[cols_to_fix] = client_df[cols_to_fix].replace([np.inf, -np.inf], np.nan).fillna(0)

# 5. Date-Based Feature: Contract Maturity
# Converting strings to datetime and calculating tenure in months
date_cols = ['date_activ', 'date_end', 'date_modif_prod', 'date_renewal']
for col in date_cols:
    client_df[col] = pd.to_datetime(client_df[col])

client_df['tenure_months'] = ((client_df['date_end'] - client_df['date_activ']).dt.days / 30).astype(int)

# 6. Time since last modification (Recency)
# Using the most recent date in the dataset as the reference point
ref_date = client_df['date_modif_prod'].max()
client_df['months_since_modification'] = ((ref_date - client_df['date_modif_prod']).dt.days / 30).fillna(0).astype(int)

# Display the new features
client_df[['id', 'consumption_shock_12m', 'forecast_gap_energy', 'tenure_months']].head()

,id,consumption_shock_12m,forecast_gap_energy,tenure_months
0,24011ae4ebbe3035111d65fa7c15bc57,0.000000,0.000000,36
1,d29c2c54acc38ff3c0614d0a653813dd,0.000000,-23.532772,85
2,764c75f661154dac3a6c254cd082ea7d,0.000000,-10.342786,73
3,bba03439a292a1e166f80264c16191cb,0.000000,-5.598900,73
4,149d57cf92fc41cf94415803a877cb4b,1.426441,-8.927089,74
